## Cabecera
Montero Barraza Álvaro David

5BV1

Ingenieria en Inteligencia Artificial 


Práctica 5. Analisis de sentimientos

A lo largo de esta practica, utilizaremos diversos modelos para el analisis de sentimientos, utilizando modelos de machine learning, diccionarios y redes neuronales. Ademas de que realizaremos un analisis de una coleccion de reseñas de comida fina en amazon y finalmente compararemos los distintos modelos al ponerlos a prueba con el dataset normalizado y vectorizado segun el modelo que corresponda

In [2]:
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from copy import deepcopy
import re
import random
from collections import Counter
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
from nltk.corpus import opinion_lexicon
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('opinion_lexicon')
nltk.download('punkt')
from sklearn.model_selection import StratifiedKFold
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
import pysentiment2 as ps
from sklearn.preprocessing import OneHotEncoder

/home/alvaromontero/.local/lib/python3.10/site-packages/cupy/_environment.py:447: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda113, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
/home/alvaromontero/.local/lib/python3.10/site-packages/cupy/_environment.py:447: UserWarning: 
------------

## **Funciones**

In [3]:
def randomize(lista):
    randomized = lista[:]  
    random.shuffle(randomized)
    return randomized

# Elimina etiquetas HTML y comentarios dentro del propio texto como indicaciones extra entre parentesis o comentarios entre --
def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-zA-Z\s.,]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc
    
# Removemos stop words unicamente en ingles
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Lematizacion mediante spacy
nlp = spacy.load("en_core_web_trf")
def lemmatize(doc):
    docs = list(nlp.pipe([doc]))
    lemas = ' '.join([token.lemma_ for doc in docs for token in doc])
    return lemas

def leer_datos(path):
    data = []
    with open(path, 'r', encoding='utf-8') as archivo: 
        reader = csv.reader(archivo)
        header = next(reader, None)  # Obtener la primera línea como encabezado
        for linea in reader:
            data.append(linea)

    return header, data

def put_label(stars):
  if stars<=2:
    return 0
  elif stars==3:
    return 1
  else:
    return 2


/home/alvaromontero/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alvaromontero/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/alvaromontero/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Funciones de vectorización

In [4]:
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm
def vectorizar_tfidf(text):
    # TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(text)
    
    # palabras únicas
    feature_names_tfidf = vectorizer.get_feature_names_out()

    print("Tamaño de Palabras únicas:", len(feature_names_tfidf))
    return tfidf_matrix

def vectorizar_onehot(input):
    # One-Hot Encoding
    vectorizer = CountVectorizer(binary=True)
    onehot_matrix = vectorizer.fit_transform(input)
    return onehot_matrix

def preprocess_texts(texts):
    processed_texts = []
    doc = list(tqdm(nlp.pipe(texts,disable=['ner','parser']),total=len(texts)))
    for text in doc:
        
        tokens = []
        for token in text:
            # Lematización
            lemma = token.lemma_
            # Stemming
            stem = stemmer.stem(lemma)
            tokens.append(stem)
        processed_texts.append(' '.join(tokens))

    return processed_texts

Diccionarios de sentimientos


In [29]:
def hiv4_polarity(text,hiv4):
    tokens = hiv4.tokenize(text)  
    score = hiv4.get_score(tokens)

    polarity = score['Polarity']
    if polarity > 0.33:
        return 2
    elif polarity < -0.33:
        return 0
    else:
        return 1

def analyze_sentiment(text):
    # Tokenizar el texto
    tokens = word_tokenize(text.lower())

    # Obtener palabras positivas y negativas del Opinion Lexicon
    pos_words = set(opinion_lexicon.positive())
    neg_words = set(opinion_lexicon.negative())

    # Calcular el sentimiento
    pos_score = sum(word in pos_words for word in tokens)
    neg_score = sum(word in neg_words for word in tokens)

    # Determinar la orientación general
    if pos_score > neg_score:
        return 1
    elif pos_score < neg_score:
        return 0
    else:
        return 2

## Adquisición de datos

In [5]:
header,raw_dataset=leer_datos("/mnt/d/Descargas/Reviews/Reviews.csv")

## Análisis exploratorio de los datos

In [6]:
print("hay",len(raw_dataset),"datos. Cada uno con",len(header),"caracteristicas")
print("columnas:")
print(header,"\n")
print(raw_dataset[1], "\n")
print(raw_dataset[2])

hay 568454 datos. Cada uno con 10 caracteristicas
columnas:
['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'] 

['2', 'B00813GRG4', 'A1D87F6ZCVE5NK', 'dll pa', '0', '0', '1', '1346976000', 'Not as Advertised', 'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".'] 

['3', 'B000LQOCH0', 'ABXLMWJIXXAIN', 'Natalia Corres "Natalia Corres"', '1', '1', '4', '1219017600', '"Delight" says it all', 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion,

In [6]:
columnas = [6,8,9]
reviews = [[row[i] for i in columnas] for row in raw_dataset]
print(reviews[2])#Segmentacion, guardamos score, summary y text

['4', '"Delight" says it all', 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.']


In [7]:
#Contamos cuantos vectores hay por clase
#Colocamos el label correcto
label_counter = [0,0,0]
for i in range(len(reviews)):
    if reviews[i][0] == '1' or reviews[i][0] == '2':#Mala opinion
        reviews[i][0] = 0
        label_counter[0] += 1
    elif reviews[i][0] == '3':#Opinion regular
        reviews[i][0] = 1
        label_counter[1] += 1
    else:
        reviews[i][0] = 2#Opinion buena
        label_counter[2] += 1
print(label_counter)

[82037, 42640, 443777]


In [8]:
def randomize(lista):
    randomized = lista[:]  
    random.shuffle(randomized)
    return randomized
reviews = randomize(reviews)
print(reviews[0])

[2, 'Delicious tea with taste of Matcha', "My husband and I love the Matcha green tea shots at Jamba juice as well as regular green tea. When I saw this tea in the local grocery store I thought I'd give it a try.<br /><br />It isn't as strong as I hoped it might be, but definitely has the Match flavor.<br /><br />It's a wonderful tea to drink at any time of the day. Light in flavor...can't really accidentally oversteep to bitterness the way you can with other greens).<br /><br />Overall, I'd recommend this tea, though it could be a tad less expensive."]


## Preprocesamiento

In [9]:

#Balance de clases
reviews_compressed = []
label_check = [0,0,0]
for i in range(len(reviews)):
    if label_check[0] == label_check[1] == label_check[2] == 42640:
        break
    if reviews[i][0] == 0 and label_check[0] < 42640:
        reviews_compressed.append(reviews[i])
        label_check[0] += 1
    elif reviews[i][0] == 1 and label_check[1] < 42640:
        reviews_compressed.append(reviews[i])
        label_check[1] += 1
    elif reviews[i][0] == 2 and label_check[2] < 42640:
        reviews_compressed.append(reviews[i])
        label_check[2] += 1
print(label_check)
print(len(reviews_compressed))
reviews = reviews_compressed

[42640, 42640, 42640]
127920


In [10]:
#Prescindimos de todo menos el score y el summary
X=[str(text[1]) for text in reviews]
y=[label[0] for label in reviews]
print(X[0])


Delicious tea with taste of Matcha


In [11]:
#Procesamiento del texto
nlp=spacy.load('en_core_web_trf')
X=[text.lower() for text in X]
X=[eliminar_etiquetas(text) for text in X]
X=[remover_apostrofes(text) for text in X]
X=[remover_especiales(text) for text in X]
X=[dobles_espacios(text) for text in X]

X_wS=deepcopy(X)

X=[remove_stopwords(text) for text in X]
X_WoS=deepcopy(X)


In [18]:
#Tokenizacion de reseñas
X_wS=preprocess_texts(X_wS)
        



100%|██████████| 127920/127920 [10:38<00:00, 200.50it/s]


In [19]:
X_WoS=preprocess_texts(X_WoS)

100%|██████████| 127920/127920 [08:19<00:00, 256.24it/s]


In [12]:
print(len(X_WoS))
print(X_WoS[10])
tfids_w=vectorizar_tfidf(X_WoS)#Vectorización a texto sin stopwords
one_hot_w=vectorizar_onehot(X_WoS)

127920
great taste .
Tamaño de Palabras únicas: 16800


In [13]:
print(tfids_w[0])

  (0, 3779)	0.44014662237877905
  (0, 14665)	0.3955396695250961
  (0, 14610)	0.36046950736387695
  (0, 8970)	0.7210277767890597


## Análsis de sentimientos con diccionarios

In [30]:
text_analysis = []
hiv4 = ps.HIV4()

for i in range(len(X_WoS)):
    if i % 12000 == 0:
        print("Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)")
    harvard_pol_text = hiv4_polarity(X_WoS[i],hiv4)
    nltk_pol_text = analyze_sentiment(X_WoS[i])
    text_analysis.append([harvard_pol_text,nltk_pol_text])

Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)


In [31]:
print(text_analysis[:10])   

[[1, 1], [2, 2], [2, 2], [0, 2], [2, 2], [2, 2], [2, 1], [0, 2], [0, 1], [2, 1]]


In [32]:
accuracy_dict = [0,0]
for i in range(len(text_analysis)):
    if y[i] == text_analysis[i][0]:
        accuracy_dict[0] += 1
    if y[i] == text_analysis[i][1]:
        accuracy_dict[1] += 1
        
accuracy_dict[0] = accuracy_dict[0] / 120000
accuracy_dict[1] = accuracy_dict[1] / 120000

In [33]:
print("Presicion usando diccionarios")
print("HIV4 ------------------ Opinion Lexicon")
print(accuracy_dict)

Presicion usando diccionarios
HIV4 ------------------ Opinion Lexicon
[0.43783333333333335, 0.35923333333333335]


## Análisis de sentimientos con Machine Learning



In [14]:
label_encoder=LabelEncoder()
label_dict1=label_encoder.fit_transform(y)
y=np.array(y)
y = tf.keras.utils.to_categorical(y, 3)

In [15]:
print(y[45])

[0. 0. 1.]


In [20]:

tfidf_array = tfids_w.toarray()
print(tfidf_array.shape[0])
batch_size=5330
aux=5330
tfidf_tensor=[]
for i in range(24):
    tfidf_tensor= tf.convert_to_tensor(tfidf_array[i*batch_size:aux-1], dtype=tf.float32)+tfidf_tensor
    aux+=batch_size

127920


2024-06-24 02:15:51.465104: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} required broadcastable shapes [Op:AddV2] name: 

In [17]:
print(len(tfidf_tensor))

5329


In [79]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_tensor,y,random_state=42)




NameError: name 'tfidf_tensor' is not defined

In [78]:
print(str(X_train[32]) + str(y_train[32]))

  (0, 7390)	0.6719517248376479
  (0, 10132)	0.7405949496774266[0. 1. 0.]


In [74]:
#Modelo de regresion logistica
print(tfids_w[1].shape)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

(1, 12466)


In [75]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitorear la pérdida de validación
    patience=10,         # Número de épocas sin mejora antes de detener el entrenamiento
    restore_best_weights=True  # Restaurar los mejores pesos al finalizar el entrenamiento
)

history = model.fit(X_train, y_train, epochs=1000, batch_size=32,callbacks=[early_stopping])

2024-06-24 01:32:29.221576: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at serialize_sparse_op.cc:388 : INVALID_ARGUMENT: indices[4] = [3,4877] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.




InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[4] = [3,4877] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse] name: 